In [0]:
orders_df = spark.read.format('parquet').load('abfss://bronze@azadlsgen7428.dfs.core.windows.net/orders')

In [0]:
orders_df.display()

In [0]:
orders_df.printSchema()

In [0]:
df = orders_df.withColumnRenamed('_rescued_data','rescued_data')

In [0]:
df.display()

In [0]:
from pyspark.sql.functions import *
from pyspark.sql.types import *
from pyspark.sql.window import Window


In [0]:
df = df.withColumn('order_date',to_timestamp(col('order_date')))
df.display()
df.printSchema()

In [0]:
df = df.withColumn('year',year(col('order_date')))
df.display()

In [0]:
df1 = df.withColumn('flag',dense_rank().over(Window.partitionBy('year').orderBy(col('total_amount').desc())))
df1.display()


In [0]:
df2 = df1.withColumn('flag_rank',rank().over(Window.partitionBy('year').orderBy(col('total_amount').desc())))
df2.display()

In [0]:
df3 = df2.withColumn('flag_row_number',row_number().over(Window.partitionBy('year').orderBy(col('total_amount').desc())))
df3.display()

In [0]:
class windows:

    def dense_rank(self,df):
        df_dense_rank = df.withColumn('flag',dense_rank().over(Window.partitionBy('year').orderBy(col('total_amount').desc())))
        return df_dense_rank
    def rank(self,df):
        df_rank = df.withColumn('flag_rank',rank().over(Window.partitionBy('year').orderBy(col('total_amount').desc())))
        return df_rank
    def row_number(self,df):
        df_row_number = df.withColumn('flag_row_number',row_number().over(Window.partitionBy('year').orderBy(col('total_amount').desc())))
        return df_row_number

In [0]:
obj = windows()
df_dense_rank = obj.dense_rank(df)
df_rank = obj.rank(df)
df_row_number = obj.row_number(df)
df_dense_rank.display()
df_rank.display()
df_row_number.display()


In [0]:
df.write.format('delta').mode('overwrite').save('abfss://silver@azadlsgen7428.dfs.core.windows.net/orders')

In [0]:
%sql

create table if not exists `ete-project`.silver.orders
using delta
location 'abfss://silver@azadlsgen7428.dfs.core.windows.net/orders'
